In [1]:
from jax import config
from dal_utils import print_result
config.update("jax_enable_x64", True)

In [2]:
import numpy as np
import numpy as np
from numpy.random import randn

rate, div, vol, spot, strike = 0.05, 0.03, 0.15, 100.0, 120.0

def create_model():
    return np.array([rate, div, vol, spot, strike])

n_paths = 2 ** 20
T = 3.0

In [3]:
default_model = create_model()

## 0. DAL
------------------

In [4]:
# !pip install dal-python -U

from dal import *
import dal
dal.__version__

'2024.05.08'

In [5]:
today = Date_(2022, 9, 15)
EvaluationDate_Set(today)

maturity = today.AddDays(int(365 * T))

In [6]:
%%time
event_dates = ["STRIKE", maturity]
events = [f"{strike}", f"call pays MAX(spot() - STRIKE, 0.0)"]

product = Product_New(event_dates, events)
model = BSModelData_New(spot, vol, rate, div)

# only price
res = MonteCarlo_Value(product, model, n_paths, "sobol", False, False)
dict(res)

CPU times: total: 297 ms
Wall time: 18.6 ms


{'PV': 5.201768833210824}

In [7]:
%%time

# price with derivatives
res = MonteCarlo_Value(product, model, n_paths, "sobol", False, True)
dict(res)

CPU times: total: 281 ms
Wall time: 22.8 ms


{'PV': 5.201768833617434,
 'd_STRIKE': -0.2358448008636096,
 'd_div': -100.50943481175733,
 'd_rate': 84.90412831090501,
 'd_spot': 0.3350314493725247,
 'd_vol': 59.58503275989193}

## 1. DAL.JAX
----------------

In [8]:
import jax
import jax.numpy as jnp

In [9]:
def compute_call_price_jax(model, T = 3.0, M=n_paths, key=jax.random.PRNGKey(1)):
    """
    Estimate the price of the call option using Monte Carlo.
    """
    # Set up
    μ, d, σ, S, K = model
    s = jnp.full(M, jnp.log(S))
    Z = jax.random.normal(key, (1, M))
    s = s + (μ - d - 0.5 * σ * σ) * T +  σ * jnp.sqrt(T) * Z[0, :]
    expectation = jnp.mean(jnp.maximum(jnp.exp(s) - K, 0))
    return jnp.exp(-μ * T) * expectation

compute_call_price_jax_jit = jax.jit(compute_call_price_jax)
_ = compute_call_price_jax(default_model).block_until_ready()
_ = compute_call_price_jax_jit(default_model).block_until_ready()

In [10]:
%%time

# only price without jit
res = compute_call_price_jax(default_model).block_until_ready()
print_result(res)

CPU times: total: 31.2 ms
Wall time: 18.5 ms


{'PV': 5.212623975501859}

In [11]:
%%time

# only price with jit
compute_call_price_jax_jit(default_model).block_until_ready()

CPU times: total: 0 ns
Wall time: 12.1 ms


Array(5.21262398, dtype=float64)

In [12]:
compute_call_value_and_grad = jax.value_and_grad(compute_call_price_jax)
compute_call_value_and_grad_jit = jax.value_and_grad(jax.jit(compute_call_price_jax))

_ = compute_call_value_and_grad(default_model)[0].block_until_ready()
_ = compute_call_value_and_grad_jit(default_model)[0].block_until_ready()

In [13]:
%%time

# price and derivatives without jit
result = compute_call_value_and_grad(default_model)
result[0].block_until_ready()
result[1].block_until_ready()
print_result(result)

CPU times: total: 0 ns
Wall time: 52.4 ms


{'PV': 5.212623975501859,
 'd_rate': 85.07049516715765,
 'd_div': -100.70836709366321,
 'd_vol': 59.709451070052516,
 'd_spot': 0.3356945569788774,
 'd_STRIKE': -0.23630693101988237}

In [14]:
%%time

# price and derivatives with jit
result = compute_call_value_and_grad_jit(default_model)
result[0].block_until_ready()
result[1].block_until_ready()
print_result(result)

CPU times: total: 0 ns
Wall time: 19.2 ms


{'PV': 5.212623975501858,
 'd_rate': 85.07049516715765,
 'd_div': -100.70836709366321,
 'd_vol': 59.709451070052516,
 'd_spot': 0.3356945569788774,
 'd_STRIKE': -0.23630693101988237}